**R181603K  NGONDZASHE L CHIKUVADZE**

**R182562C  ONWELL J SHOKO**

In [25]:
!pip install pyngrok

In [26]:
!pip install -q streamlit

In [27]:
from pyngrok import ngrok

In [37]:
!ngrok authtoken 1npJviy2rm2H1RzUbevBUv2GLGw_ep9y73PeZMfA5zeSBzzs

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [38]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
!pip install -q python-crfsuite

In [40]:
!pip install -q sklearn-pycrfsuite

In [41]:
%%writefile qg.py
import streamlit as st
import string
import pycrfsuite
import nltk
from nltk.corpus import stopwords
from nltk.tag import CRFTagger
from nltk import word_tokenize,sent_tokenize
import pandas as pd
import numpy as np
import spacy
import time
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

ct = CRFTagger()
nlp = spacy.load('en')

PAGE_CONFIG = {"page_title":"StColab.io", "page_icon":":smiley:","layout":"centered"}
st.set_page_config("PAGE_CONFIG")

st.title("Deep Learning - Question Generation")
st.header("R181603K")
st.header("R182562C")

def main():
  train_set  = pd.read_csv('/content/drive/My Drive/Colab Notebooks/kbs_project/Train_and_test_data/train.txt',sep=' ',names=['word','Brill','tag']).drop('Brill',1)
  train_set = [[tuple(x) for x in train_set.values]]
  sentence = ' '.join(open('/content/drive/My Drive/Colab Notebooks/kbs_project/Text_Passages2.txt','r').readlines()).rstrip("\n")

  def process_text(passage):
    exclude_set = set(['“','”',':'])
    no_punctuation = [char for char in passage if char not in string.punctuation + "“”."]
    no_punctuation = ''.join(no_punctuation)
    no_punctuation = [word for word in no_punctuation.split() if word.lower() not in stopwords.words('english')]
    return no_punctuation

  nltk.download('stopwords')
  sentence = process_text(sentence)
  sentence = ' '.join(sentence)  
  nltk.download('punkt')

  sentences = sent_tokenize(sentence)
  word_list = [[]]
  word_list.clear()
  wlist = []
  i = 0
  j = 0

  for sent in sentences:
      for word in sent.split():
          wlist.append(word)
          j += 1
      wlist.copy
      word_list.append(wlist.copy())
      wlist.clear()
      i += 1

  ct.set_model_file(model_file='/content/drive/My Drive/Colab Notebooks/kbs_project/model.crf.tagger')
  word_list = ct.tag_sents(word_list)

  test_set = pd.read_csv('/content/drive/My Drive/Colab Notebooks/kbs_project/Train_and_test_data/test.txt', sep=' ', names=['words','Brill','tag'], index_col=False).drop(['Brill','tag'],1)
  test_set = test_set.values.tolist()
  ct.tag_sents(test_set)

  test_setEval = pd.read_csv('/content/drive/My Drive/Colab Notebooks/kbs_project/Train_and_test_data/test.txt', sep=' ', names=['words','Brill','tag']).drop('Brill',1)
  test_setEval = [[tuple(x) for x in test_setEval.values]]
  ct.evaluate(test_setEval)
  word_list = np.array(word_list)
  word_list = np.reshape(word_list, (-1,2))

  def findSubject(pattern):
      if pattern[0] == 'NP' and pattern[1] == 'VP' and pattern[2] == 'NP':
          return True
      else:
          return False
  pattern = []
  subjects = []
  word_list_length = len(word_list)

  for i in range(0, len(word_list)):
      try:
          if((word_list_length - i == 2)):
              break
          else:
              pattern = [word_list[i][1].split('-')[1], word_list[i+1][1].split('-')[1], word_list[i+2][1].split('-')[1]]
              if findSubject(pattern):
                  #print('Potential Subject Found at index {}'.format(i))
                  #print('Subject -> {} => {} {}'.format(word_list[i][0], word_list[i+1][0], word_list[i+2][0]))
                  sub = [word_list[i][0],word_list[i+1][0],word_list[i+2][0]]
                  subjects.append(' '.join(sub))
                  print('--------------------')
      except IndexError:
          #print("Out of Index")
          break;

  subjects = np.array(subjects)
  subjects = np.reshape(subjects, (-1,1))
  refferingDataFrame = pd.DataFrame(columns=('Word_POS-TAG_Person','nullColumn'))

  for i in range(0,len(subjects)):
      for j in range(0,len(subjects[i][0].split(" "))):
          wordBag = subjects[i][0].split(" ")
          pos = ct.tag([wordBag[j]])
          person = [ent.label_ for ent in nlp(pos[0][0]).ents]
          person = ' '.join(person)
          if not person:
              person = ""
          refferingDataFrame.loc[len(refferingDataFrame)] = [(pos[0][0] +"_"+ pos[0][1].split('-')[1] +"_"+ person).rstrip('_'), ""]

  lstm = nn.LSTM(3, 3)
  inputs = [autograd.Variable(torch.randn(1, 3)) for _ in range(5)]
  hidden = (autograd.Variable(torch.randn(1, 1, 3)), autograd.Variable(torch.randn(1, 1, 3)))

  for i in inputs:
      out, hidden = lstm(i.view(1, 1, -1), hidden)
      
  inputs = torch.cat(inputs).view(len(inputs), 1, -1)
  hidden = (autograd.Variable(torch.randn(1, 1, 3)), autograd.Variable(torch.randn(1, 1, 3)))
  out, hidden = lstm(inputs, hidden)
  #print(out)
  #print(hidden)

  def prepare_sequence(seq, to_ix):
      idxs = [to_ix[w] for w in seq]
      tensor = torch.LongTensor(idxs)
      return autograd.Variable(tensor)
  unprocessed_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/kbs_project/Train_and_test_data/LSTM_train_set.csv',header=None)

  t_data_for_lstm = []
  training_data = []
  for phrase in unprocessed_data.itertuples():
      t_data_for_lstm.append(list(zip([[phrase[1]]], [[phrase[2]]])))

  for i in range(0, len(t_data_for_lstm)):
      training_data.append(t_data_for_lstm[i][0])
      
  word_to_ix = {}
  tag_to_ix = {}

  for sent, tags in training_data:
      for word in sent:
          if word not in word_to_ix:
              word_to_ix[word] = len(word_to_ix)
      for word1 in tags:
          if word1 not in tag_to_ix:
              tag_to_ix[word1] = len(tag_to_ix)
                              
  EMBEDDING_DIM = 6
  HIDDEN_DIM = 6

  # The LSTM model
  class LSTMTagger(nn.Module):
      def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
          super(LSTMTagger, self).__init__()
          self.hidden_dim = hidden_dim
          self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
          self.lstm = nn.LSTM(embedding_dim, hidden_dim)
          self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
          self.hidden  = self.init_hidden()
          
      def init_hidden(self):
          return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)), autograd.Variable(torch.zeros(1, 1, self.hidden_dim)))
          
      def forward(self, sentence):
          embeds = self.word_embeddings(sentence)
          lstm_out, self.hidden = self.lstm(embeds.view(len(sentence), 1, -1), self.hidden)
          tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
          tag_scores = F.log_softmax(tag_space)
          return tag_scores
      
  model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
  loss_function = nn.NLLLoss()
  optimizer = optim.SGD(model.parameters(), lr=0.1)

  inputs = prepare_sequence(training_data[0][0], word_to_ix)
  tag_scores = model(inputs)
  t0 = time.time()
  iterations = 300
  progress_after = iterations/4

  # Training the RNN for 300 iterations
  for epoch in range(0,iterations):
      for sentence, tags in training_data:
          model.zero_grad()
          model.hidden = model.init_hidden()
          sentence_in = prepare_sequence(sentence, word_to_ix)
          targets = prepare_sequence(tags, tag_to_ix)
          if(progress_after == epoch):
              #print("Training {}% Completed".format((progress_after / iterations) * 100))
              progress_after = progress_after + (iterations / 4)
              
          tag_scores = model(sentence_in)
          
          loss = loss_function(tag_scores, targets)
          loss.backward()
          optimizer.step() 
  #print("Training Complete!\nTotal Training time :", round(time.time()-t0, 2), "s\n")
       
  inputs = prepare_sequence(training_data[0][0], word_to_ix)
  tag_scores = model(inputs)
  def generate_questions(model, ref_data_frame):
      question = ""
      sentence = ""
      index = 0
      for index, word_phrase in ref_data_frame.iterrows():
          try:
              inputs = prepare_sequence([word_phrase[0]], word_to_ix)
              tag_scores = model(inputs)
          except KeyError:
              # ignore
              pass

          maxVal = max(tag_scores.data.numpy()[0])
          index_loc = 0
          probability_tag_scores = tag_scores.data.numpy().ravel()

          for i in range(0, len(probability_tag_scores)):
              if (probability_tag_scores.ravel()[i] == maxVal):
                  index_loc = i

          for key, value in tag_to_ix.items():
              if (value == index_loc):
                  sentence = word_phrase[0].split('_')
                  if "NP" in sentence:
                      question = key.replace("NP", sentence[0])
                      st.text(question)
                  elif "VP" in sentence:
                      if "NP" in key and "N1" in key:
                          index = [i for i,j in enumerate(subjects.ravel()) if sentence[0] in j]
                          ref_subject = subjects.ravel()[index][0].split()
                          question = key.replace("NP", ref_subject[0]) 
                          question = question.replace("N1",ref_subject[-1])
                          st.text(question)
                      else:
                          index = [i for i,j in enumerate(subjects.ravel()) if sentence[0] in j]
                          ref_subject = subjects.ravel()[index][0].split()
                          question = key.replace("NP", ref_subject[0])
                          st.text(question)

  generate_questions(model, refferingDataFrame)  

if __name__ == '__main__':
  main()

Overwriting qg.py


In [42]:
!streamlit run --server.port 80 qg.py&>/dev/null&

In [43]:
url = ngrok.connect(port=8501)

In [44]:
print(url)

NgrokTunnel: "http://e0bb97c73fef.ngrok.io" -> "http://localhost:80"


In [35]:
ngrok.kill()